In [190]:
from bs4 import BeautifulSoup
import requests
import re
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchFrameException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
url = 'https://www.cars-data.com/robots.txt'
response  = requests.get(url)
print(response.text)

User-agent: *
Allow: /

Sitemap: https://www.cars-data.com/en/val-en.xml
Sitemap: https://www.cars-data.com/en/en-index.xml	
Sitemap: https://www.cars-data.com/fr/fr-index.xml	
Sitemap: https://www.cars-data.com/de/de-index.xml	
Sitemap: https://www.cars-data.com/no/no-index.xml	
Sitemap: https://www.cars-data.com/es/es-index.xml	
Sitemap: https://www.cars-data.com/cz/cz-index.xml	
Sitemap: https://www.cars-data.com/hu/hu-index.xml	
Sitemap: https://www.cars-data.com/ro/ro-index.xml	
Sitemap: https://www.cars-data.com/srb/srb-index.xml	
Sitemap: https://www.cars-data.com/jp/jp-index.xml	
Sitemap: https://www.cars-data.com/hi/hi-index.xml	
Sitemap: https://www.cars-data.com/za/za-index.xml	
Sitemap: https://www.cars-data.com/se/se-index.xml
Sitemap: https://www.cars-data.com/el/el-index.xml

User-agent: *
Disallow: /cn/


Create a webdriver and connect to single car page.

In [238]:
from fake_useragent import UserAgent
from selenium.webdriver import Firefox

url = 'https://www.cars-data.com/en/audi-a6-2-7-tdi-advance-specs/3835'
ua = UserAgent()
user_agent = ua.random


profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", user_agent)
#profile.set_preference("dom.disable_beforeunload", True) #dom.disable_open_during_load", False
driver = webdriver.Firefox(profile)

driver.get(url)
#response  = requests.get(url, headers = user_agent)

Click on button view more specs to expand scrapable specs

In [239]:
view_specs_button = driver.find_element_by_xpath('//a[contains(@class, "viewphotosbutton")]')
view_specs_button.click()

Switch frame to ad and click the close button

In [240]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchFrameException

# try:
#     driver.switch_to.frame("apple")
# except NoSuchElementException:  #spelling error making this code not work as expected
#     pass

iframe = ["ad_iframe", "aswift_9"]

try:
    driver.switch_to.frame(iframe[1])
    view_specs_button = driver.find_element_by_xpath('//div[(@id="dismiss-button")]')
    view_specs_button.click()
    print('found aswift_9')
except (NoSuchElementException, NoSuchFrameException) as E:
    driver.switch_to.frame(iframe[0])
    print('found ad_frame')
    view_specs_button = driver.find_element_by_xpath('//div[(@id="dismiss-button")]') 
    view_specs_button.click()
#'//a[contains(@class, "viewphotosbutton")]'

found ad_frame


Switch frame back to default page

In [241]:
driver.switch_to.default_content()

Covert the expanded page into a BeautifulSoup object

In [242]:
soup = BeautifulSoup(driver.page_source, 'lxml')
driver.close()

In [243]:
print(soup.prettify())

<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <script src="https://www.googletagservices.com/activeview/js/current/osd.js">
  </script>
  <script src="https://partner.googleadservices.com/gampad/cookie.js?domain=www.cars-data.com&amp;callback=_gfp_s_&amp;client=ca-pub-0524013268692551&amp;cookie=ID%3D47d5b57b50d6a717-22d3e3de6aca00ce%3AT%3D1632434737%3ART%3D1632434737%3AS%3DALNI_MZsfzKXT_vHaAwUMYgQgYii0R-T1A">
  </script>
  <script async="" src="//cse.google.com/adsense/search/async-ads.js">
  </script>
  <script id="google_shimpl" src="https://pagead2.googlesyndication.com/pagead/managed/js/adsense/m202109210101/show_ads_impl.js?bust=31062896">
  </script>
  <script async="" data-callback="put-your-callback-macro-here" data-target="" id="CleverCoreLoader49605" src="//scripts.cleverwebserver.com/c17f34697eff61933db1f1b27045649c.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   setTimeout(function downloadJsA

This function will find the next object if part of the given text is in the proceeding object

In [244]:
def get_car_value(soup, field_name):
    
    '''Grab a value from Box Office Mojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in
    the next sibling object (the value for that attribute) or None if nothing is found.
    '''
    
    obj = soup.find(text=re.compile(field_name))
    
    if not obj: 
        return None
    
    # this works for most of the values
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

Find car name

In [245]:
name = soup.find(class_ = "col-10").find('h1').text

print(name)

Audi A6 2.7 TDI Advance technical specs


Find car make year

In [246]:
year = get_car_value(soup, 'Introduction:')

print(year)

june 2010


Find car price

In [247]:
price = get_car_value(soup, "Price:")

print(price)

€ 56.170


Find car body style

In [13]:
body_type = get_car_value(soup, 'Body Type:')

print(body_type)

5-doors, hatchback


Find car transmission type

In [14]:
transmission = get_car_value(soup, 'Transmission:')

print(transmission)

5 speed manual transmission


Find car drivetrain type

In [15]:
drivetrain = get_car_value(soup, 'Drive Wheel :')

print(drivetrain)

front


Find car fuel type

In [16]:
fuel = get_car_value(soup, "Fuel Type:")
print(fuel)

diesel


Find car horsepower

In [17]:
power = get_car_value(soup, "Max Power:")
print(power)

66 kw (90 hp)


Find number of cylinders in engine

In [18]:
cylinders = get_car_value(soup, 'Cylinders:')

print(cylinders)

4, in line


Find car engine capacity

In [19]:
engine_cap = get_car_value(soup, "Engine Capacity:")

print(engine_cap)

1598 cc


Find car bore-stroke dimensions

In [20]:
bore_stroke = get_car_value(soup, "Bore X Stroke:")

print(bore_stroke)

79,5 x 80,5 mm


Find car compression ratio

In [21]:
compression = get_car_value(soup, "Compression Ratio:")

print(compression)

16,5 :1


Find car torque output

In [22]:
torque = get_car_value(soup, "Max Torque:")

print(torque)

230 nm


Find car max speed

In [23]:
speed = get_car_value(soup, "Top Speed:")

print(speed)

182 km/h


Find car acceleration 

In [24]:
acceleration = get_car_value(soup, "Acceleration 0-100 Km / H:")

print(acceleration)

11,6 s


Place scrapped data into dictionary

In [25]:
headers = ['name', 'year', 'price','body type', 'transmission',
          'drivetrain', 'fuel', 'power', 'cylinders', 'engine cap',
          'bore stroke', 'compression', 'torque', 'speed', 'acceleration']

car_dict = dict(zip(headers,[name, year, price, body_type, transmission,
                            drivetrain, fuel, power, cylinders, engine_cap,
                            bore_stroke, compression, torque, speed, acceleration]))

print(car_dict)

{'name': 'Audi A1 Sportback 1.6 TDI 90hp Attraction Pro Line technical specs', 'year': 'january 2012', 'price': '€ 24.380', 'body type': '5-doors, hatchback', 'transmission': '5 speed manual transmission', 'drivetrain': 'front', 'fuel': 'diesel', 'power': '66 kw (90 hp)', 'cylinders': '4, in line', 'engine cap': '1598 cc', 'bore stroke': '79,5 x 80,5 mm', 'compression': '16,5 :1', 'torque': '230 nm', 'speed': '182 km/h', 'acceleration': '11,6 s'}


Read specific site map page

In [604]:

import random
url = 'https://www.cars-data.com/en/types-en-45.xml'
ua = UserAgent()
user_agent = {'user-agent': ua.random}
response  = requests.get(url, headers = user_agent)
soup=BeautifulSoup(response.text,'lxml')

jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [ ]:
print(soup.prettify())

Find all urls on site map page

In [605]:
data = soup.find_all(text=re.compile('https'))

Randomly select 25 cars from site map

In [606]:
samp = random.sample(data,25)

Create and empty car list

In [574]:
car_list = []

Append urls to a clean list

In [607]:
# data = soup.find_all('xhtml:link')
# data= data.get_text()


for link in samp:
    car_list.append(link.text)
    
for row in car_list:
    print(row)

https://www.cars-data.com/en/skoda-superb-2-0-tsi-4x4-greentech-style-business-specs/71550
https://www.cars-data.com/en/smart-fortwo-52kw-prime-specs/67973
https://www.cars-data.com/en/subaru-impreza-plus-2-0r-awd-specs/46530
https://www.cars-data.com/en/ssangyong-korando-2-0-xdi-sapphire-4wd-specs/71663
https://www.cars-data.com/en/suzuki-alto-1-0-gl-specs/46849
https://www.cars-data.com/en/subaru-legacy-outback-2-5-awd-specs/46747
https://www.cars-data.com/en/suzuki-baleno-1-3-gs-specs/46908
https://www.cars-data.com/en/suzuki-alto-1-0-twist-specs/46881
https://www.cars-data.com/en/suzuki-alto-1-1-glx-specs/46888
https://www.cars-data.com/en/ssangyong-korando-2-0-quartz-2wd-specs/71649
https://www.cars-data.com/en/ssangyong-rexton-w-xdi-4wd-executive-specs/72662
https://www.cars-data.com/en/ssangyong-rexton-rx-270-xdi-sport-specs/46354
https://www.cars-data.com/en/ssangyong-kyron-m-270-xdi-4wd-specs/46308
https://www.cars-data.com/en/subaru-impreza-2-0-gt-turbo-555-awd-specs/46462
ht

In [608]:
len(car_list)

225

Create a large function to take urls in car_list and automate the scrapping process from above. Large try statement to find edge cases of ad windows.

In [609]:
from selenium.webdriver.support import expected_conditions as EC
def get_car_dict(link):
    
    url = link
    ua = UserAgent()
    user_agent = ua.random

    profile = webdriver.FirefoxProfile()
    profile.set_preference("general.useragent.override", user_agent)

    driver = webdriver.Firefox(profile)
    driver.get(url)
    
    view_specs_button = driver.find_element_by_xpath('//a[contains(@class, "viewphotosbutton")]')
    view_specs_button.click()
    time.sleep(1)
    
    iframe = ["ad_iframe", "aswift_9", "aswift_10", "aswift_2"]

# view_specs_button.click()
# element = driver.find_element_by_xpath(xpath)
# driver.execute_script("arguments[0].click();", element)
# wait = WebDriverWait(driver, 10)
# element = wait.until(EC.element_to_be_clickable((By.ID, 'someid')))

    try:
        driver.switch_to.frame(iframe[1])
        view_specs_button = driver.find_element_by_xpath('//div[(@id="dismiss-button")]') 
        driver.execute_script("arguments[0].click();", view_specs_button)
        
    except (NoSuchElementException, NoSuchFrameException, ElementNotInteractableException, ElementClickInterceptedException) as E:
        try:
            driver.switch_to.frame(iframe[0])
            view_specs_button = driver.find_element_by_xpath('//div[(@id="dismiss-button")]') 
            driver.execute_script("arguments[0].click();", view_specs_button)
        
        except (NoSuchElementException, NoSuchFrameException, ElementNotInteractableException, ElementClickInterceptedException) as F:
            try:
                driver.switch_to.frame(iframe[2])
                view_specs_button = driver.find_element_by_xpath('//div[(@id="dismiss-button")]') 
                driver.execute_script("arguments[0].click();", view_specs_button)
        
            except (NoSuchElementException, NoSuchFrameException) as G:
                try:
                    driver.switch_to.frame(iframe[3])
                    view_specs_button = driver.find_element_by_xpath('//div[(@id="dismiss-button")]') 
                    driver.execute_script("arguments[0].click();", view_specs_button) 

                except (NoSuchElementException, NoSuchFrameException) as F:  
                    try:
                        driver.switch_to.frame(iframe[0])
                        view_specs_button = driver.find_element_by_xpath('//div[(@id="dismiss-button")]') 
                        driver.execute_script("arguments[0].click();", view_specs_button)
                    except (NoSuchElementException, NoSuchFrameException) as F:     
                        try:
                            pass
                        except:
                            pass
    
#     except (NoSuchElementException, NoSuchFrameException, ElementNotInteractableException, ElementClickInterceptedException) as F:
#         driver.switch_to.frame(iframe[2])
#         view_specs_button = driver.find_element_by_xpath('//div[(@id="dismiss-button")]') 
#         driver.execute_script("arguments[0].click();", view_specs_button)
    
#     except (NoSuchElementException, NoSuchFrameException, ElementNotInteractableException, ElementClickInterceptedException) as G:
#         driver.switch_to.frame(iframe[3])
#         view_specs_button = driver.find_element_by_xpath('//div[(@id="dismiss-button")]') 
#         driver.execute_script("arguments[0].click();", view_specs_button) 

#     except Exception:
#         pass
    
    driver.switch_to.default_content()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    time.sleep(1)
    driver.close()
    
    name = soup.find(class_ = "col-10").find('h1').text
    
    year = get_car_value(soup, 'Introduction:')
    
    price = get_car_value(soup, "Price:")
    
    body_type = get_car_value(soup, 'Body Type:')
    
    transmission = get_car_value(soup, 'Transmission:')
    
    drivetrain = get_car_value(soup, 'Drive Wheel :')
    
    fuel = get_car_value(soup, "Fuel Type:")
    
    power = get_car_value(soup, "Max Power:")
    
    cylinders = get_car_value(soup, 'Cylinders:')
    
    engine_cap = get_car_value(soup, "Engine Capacity:")
    
    bore_stroke = get_car_value(soup, "Bore X Stroke:")
    
    compression = get_car_value(soup, "Compression Ratio:")
    
    torque = get_car_value(soup, "Max Torque:")
    
    speed = get_car_value(soup, "Top Speed:")
    
    acceleration = get_car_value(soup, "Acceleration 0-100 Km / H:")
    
    headers = ['name', 'year', 'price','body type', 'transmission',
          'drivetrain', 'fuel', 'power', 'cylinders', 'engine cap',
          'bore stroke', 'compression', 'torque', 'speed', 'acceleration']

    car_dict = dict(zip(headers,[name, year, price, body_type, transmission,
                            drivetrain, fuel, power, cylinders, engine_cap,
                            bore_stroke, compression, torque, speed, acceleration]))
    
    wait = 1.5 + 2*random.random()
    time.sleep(wait)
    
    return car_dict

In [610]:
car_page_info_list = []

Appends each car dictionary to a list

In [611]:
for idx, link in enumerate(car_list[:]):
    car_page_info_list.append(get_car_dict(link))
    print(idx)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224


In [612]:
len(car_page_info_list)

225

Converts list of car dictionaries to a dataframe

In [613]:
import pandas as pd
car_page_df17 = pd.DataFrame(car_page_info_list)
car_page_df17.set_index('name', inplace=True)
car_page_df17

,year,price,body type,transmission,drivetrain,fuel,power,cylinders,engine cap,bore stroke,compression,torque,speed,acceleration
name,,,,,,,,,,,,,,
Skoda Superb 2.0 TSI 4x4 Greentech Style Business technical specs,june 2015,€ 53.240,"5-doors, hatchback",6 speed automatic with double clutch,front+rear,gasoline,206 kw (280 hp),"4, in line",1984 cc,"82,5 x 92,8 mm","9,3 :1",350 nm,250 km/h,"5,8 s"
Smart Fortwo 52kW Prime technical specs,january 2015,€ 15.445,"3-doors, hatchback",6 speed automatic with double clutch,rear,gasoline,52 kw (71 hp),"3, in line",999 cc,"72,2 x 81,3 mm","10,5 :1",91 nm,151 km/h,"15,1 s"
Subaru Impreza Plus 2.0R AWD technical specs,october 2005,€ 27.200,"5-doors, station wagon",5 speed manual transmission,front+rear,gasoline,118 kw (160 hp),"4, boxer",1994 cc,"92,0 x 75,0 mm","11,1 :1",186 nm,203 km/h,"8,8 s"
Ssangyong Korando 2.0 XDi Sapphire 4WD technical specs,january 2014,€ 42.853,"5-doors, suv/crossover",6 speed manual transmission,front+rear,diesel,110 kw (150 hp),"4, in line",1998 cc,"86,2 x 85,6 mm","16,5 :1",360 nm,180 km/h,"10,9 s"
Suzuki Alto 1.0 GL technical specs,november 1996,€ 7.712,"3-doors, hatchback",5 speed manual transmission,front,gasoline,40 kw (54 hp),"4, in line",993 cc,"72,0 x 61,0 mm","9,4 :1",77 nm,150 km/h,"15,5 s"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Volvo V40 T3 Ocean Race technical specs,april 2015,€ 37.695,"5-doors, hatchback",6 speed automatic,front,gasoline,112 kw (152 hp),"4, in line",1498 cc,"82,0 x 70,9 mm","10,5 :1",250 nm,210 km/h,"8,3 s"
Volvo V70 3.2 Kinetic technical specs,august 2007,€ 56.390,"5-doors, station wagon",6 speed automatic,front,gasoline,175 kw (238 hp),"6, in line",3192 cc,"84,0 x 98,0 mm","10,8 :1",320 nm,235 km/h,"8,4 s"
Volvo V60 T5 Ocean Race technical specs,may 2011,€ 44.795,"5-doors, station wagon",6 speed manual transmission,front,gasoline,177 kw (240 hp),"4, in line",1999 cc,"87,5 x 83,1 mm","10,0 :1",320 nm,230 km/h,"7,5 s"


Saves car dataframe

In [614]:
car_page_df17.to_csv('types-en-37-45.csv')

Concatenates car data to larger dataframe

In [615]:
car_page_df114 = [car_page_df113, car_page_df17]
car_page_df114 = pd.concat(car_page_df114)
car_page_df114.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1098 entries, Audi A3 1.9 TDI Ambition technical specs to Volvo V70 2.4D Summum technical specs
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          1055 non-null   object
 1   price         1098 non-null   object
 2   body type     1098 non-null   object
 3   transmission  1098 non-null   object
 4   drivetrain    1098 non-null   object
 5   fuel          1098 non-null   object
 6   power         1055 non-null   object
 7   cylinders     1093 non-null   object
 8   engine cap    1093 non-null   object
 9   bore stroke   1093 non-null   object
 10  compression   1093 non-null   object
 11  torque        1055 non-null   object
 12  speed         1098 non-null   object
 13  acceleration  1098 non-null   object
dtypes: object(14)
memory usage: 128.7+ KB


Saves larger dataframe

In [616]:
car_page_df114.to_csv('accum_data.csv')

In [617]:
df_test = pd.read_csv('accum_data.csv')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1098 entries, 0 to 1097
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          1098 non-null   object
 1   year          1055 non-null   object
 2   price         1098 non-null   object
 3   body type     1098 non-null   object
 4   transmission  1098 non-null   object
 5   drivetrain    1098 non-null   object
 6   fuel          1093 non-null   object
 7   power         1055 non-null   object
 8   cylinders     1093 non-null   object
 9   engine cap    1093 non-null   object
 10  bore stroke   1093 non-null   object
 11  compression   1093 non-null   object
 12  torque        1055 non-null   object
 13  speed         1098 non-null   object
 14  acceleration  1098 non-null   object
dtypes: object(15)
memory usage: 128.8+ KB
